# Exercise 8. Inferencing

## Inferencing
### Create scoring script

First, we will create a scoring script that will be invoked by the web service call. Note that the scoring script must have two required functions:
* `init()`: In this function, you typically load the model into a `global` object. This function is executed only once when the Docker container is started. 
* `run(input_data)`: In this function, the model is used to predict a value based on the input data. The input and output typically use JSON as serialization and deserialization format, but you are not limited to that.

Refer to the scoring script `pytorch_score.py` for this tutorial. Our web service will use this file to predict the dog breed based on a 120 class model trained earlier, located in the folder `model`. We will test the scoring file locally first before you go and deploy the web service.

1. Import the scoring script, so that init() and run() are accessible

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import pytorch_score

2. Call the init function -- this will load the model and the class_names from the model directory

In [ ]:
pytorch_score.init()

3. Add some helper functions:

In [ ]:
import os, json, base64
from io import BytesIO
import matplotlib.pyplot as plt
from skimage import io
from PIL import Image
import urllib.request
import io, requests

##Get random dog
def get_random_dog():
    r = requests.get(url ="https://dog.ceo/api/breeds/image/random")
    URL= r.json()['message']
    return URL

def imgToBase64(img):
    """Convert pillow image to base64-encoded image"""
    imgio = BytesIO()
    img.save(imgio, 'JPEG')
    img_str = base64.b64encode(imgio.getvalue())
    return img_str.decode('utf-8')

4. Find an image of a dog and call the run() function

In [ ]:
##Get Random Dog Image
URL = get_random_dog()

with urllib.request.urlopen(URL) as url:
    test_img = io.BytesIO(url.read())


plt.imshow(Image.open(test_img))

base64Img = imgToBase64(Image.open(test_img))

result = pytorch_score.run(input_data=json.dumps({'data': base64Img}))
print(URL)
print(json.loads(result))

### Register the model
Once the run completes, we can register the model that was created.

**Please use a unique name for the model**

In [ ]:
from azureml.core.model import Model
model = Model.register(ws, model_name='model', model_path = 'model', description='120 Dogbreeds')
print(model.name, model.id, model.version, sep = '\t')